<a href="https://colab.research.google.com/github/AgomferAustral/DMA-Caras/blob/main/Eigenfaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# conexion al Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cargar las librerias que vamos a utilizar

In [2]:
import cv2
import math
import numpy as np
import os
import pickle
import random
import sys
import numpy as np
from sklearn.manifold import Isomap
from PIL import Image
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
#!pip install --upgrade numpy
#!pip install --upgrade scipy

# Preprocesamos imagenes
## La carpeta de origen se llama Eigenfaces


In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Definicion de ruta de conexion - Origen - Destino imagenes

img_size=(60,60)

ruta_entrada = '/content/drive/MyDrive/Eigenfaces'
ruta_salida = f'/content/drive/MyDrive/Eigenfaces2-{img_size[0]}x{img_size[1]}'


# prompt: verificar si la carpeta en "ruta_salida" existe. Si no existe, crearla

if not os.path.exists(ruta_salida):
  os.makedirs(ruta_salida)

def procesar_archivos_en_carpetas(ruta_principal,ruta_final, imgsize):
  """Recorre las carpetas dentro de la ruta principal y procesa los archivos.

  Args:
    ruta_principal: La ruta de la carpeta principal.
  """

  fraccion = 8


  x_inicial = imgsize[0] // (fraccion - 2)
  y_inicial = imgsize[1] // (fraccion - 2)
  alto = imgsize[1]
  ancho = imgsize[0]
  multip = fraccion / (fraccion - 2)
  img_size = (math.floor(ancho * multip) , math.floor(alto * multip))


  print(f"Ruta principal: {ruta_principal}")
  for carpeta_actual, _, archivos in os.walk(ruta_principal):
    print(f"Carpeta actual: {carpeta_actual}")
    for archivo in archivos:
      print(f"Archivo: {archivo}")
      ruta_completa = os.path.join(carpeta_actual, archivo)
      try:

            img = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)

            img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)

            if img is None:
                print(f"⚠️ No se pudo leer: {ruta_completa}")
                continue

            # Detectar rostros
            faces = face_cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=10, minSize=(110, 110))

            for i, (x, y, w, h) in enumerate(faces):
                face = img[y:y+h, x:x+w]  # Recortar rostro
                face_resized = cv2.resize(face, img_size)  # Redimensionar

                face_cutted = face_resized[y_inicial:y_inicial+alto, x_inicial:x_inicial+ancho]

                # Crear carpeta de salida manteniendo la estructura original
                relative_path = os.path.relpath(carpeta_actual, ruta_entrada)
                output_folder = os.path.join(ruta_salida, relative_path)
                os.makedirs(output_folder, exist_ok=True)

                # Guardar rostro procesado
                output_path = os.path.join(output_folder, f"{os.path.splitext(archivo)[0]}_face{i}.jpg")
                cv2.imwrite(output_path, face_cutted)
                print(f"✅ Guardado: {output_path}")


      except Exception as e:
          print(f"❌ Error procesando {ruta_completa}: {e}")


      except Exception as e:
        print(f"Error al procesar el archivo {ruta_completa}: {e}")

In [4]:
# Preprocesamos imagenes (escala de grises, recortes y escalado)
print("Procesando archivos")
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size)
print("Proceso inicial terminado ...")

Procesando archivos
Ruta principal: /content/drive/MyDrive/Eigenfaces
Carpeta actual: /content/drive/MyDrive/Eigenfaces
Carpeta actual: /content/drive/MyDrive/Eigenfaces/Noelia M
Archivo: 1742907849847.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces2-60x60/Noelia M/1742907849847_face0.jpg
Archivo: 1742907849811.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces2-60x60/Noelia M/1742907849811_face0.jpg
Archivo: 20250321_140807.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces2-60x60/Noelia M/20250321_140807_face0.jpg
Archivo: 20250321_140804.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces2-60x60/Noelia M/20250321_140804_face0.jpg
Archivo: 20250321_140808.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces2-60x60/Noelia M/20250321_140808_face0.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces2-60x60/Noelia M/20250321_140808_face1.jpg
Archivo: 1742907849823.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfaces2-60x60/Noelia M/1742907849823_face0.jpg
✅ Guardado: /content/drive/MyDrive/Eigenfac

## el preprocesado queda en la carpeta Eigenfaces2-60x60

### copie una version de esa carpeta a "origen" para usar a continuacion

In [14]:
# origen de datos preprocesados
ruta_origen = "/content/drive/MyDrive/Origen"

In [15]:
# Dividimos conjunto de datos

def dividir_datos(ruta_origen, proporcion_entrenamiento=0.75):
    """
    Divide las imágenes en conjuntos de entrenamiento y prueba,
    y guarda las etiquetas usando un diccionario de listas.

    Args:
        ruta_origen (str): Ruta al directorio raíz con las subcarpetas de imágenes.
        proporcion_entrenamiento (float): Proporción de imágenes para entrenamiento.

    Returns:
        dict: Diccionario con las rutas de las imágenes y etiquetas, organizadas en entrenamiento y prueba.
    """

    conjuntos_datos = {
        "entrenamiento": {"imagenes": [], "etiquetas": []},
        "prueba": {"imagenes": [], "etiquetas": []}
    }

    for persona in os.listdir(ruta_origen):
        ruta_persona = os.path.join(ruta_origen, persona)
        if os.path.isdir(ruta_persona):
            imagenes = [os.path.join(ruta_persona, img) for img in os.listdir(ruta_persona) if img.lower().endswith(('.png', '.jpg', '.jpeg'))]
            entrenamiento, prueba = train_test_split(imagenes, train_size=proporcion_entrenamiento, random_state=73)  # Semilla para reproducibilidad

            # Guardar datos de entrenamiento
            conjuntos_datos["entrenamiento"]["imagenes"].extend(entrenamiento)
            conjuntos_datos["entrenamiento"]["etiquetas"].extend([persona] * len(entrenamiento))

            # Guardar datos de prueba
            conjuntos_datos["prueba"]["imagenes"].extend(prueba)
            conjuntos_datos["prueba"]["etiquetas"].extend([persona] * len(prueba))

    return conjuntos_datos

In [16]:
# En la variable "ruta_origen" se encuentran las imagenes preprocesadas para enternamiento
datos_divididos = dividir_datos(ruta_origen)


# Acceder a los datos
lista_nombres_imagenes_entrenamiento = datos_divididos["entrenamiento"]["imagenes"]
lista_etiquetas_entrenamiento = datos_divididos["entrenamiento"]["etiquetas"]
lista_nombres_imagenes_prueba = datos_divididos["prueba"]["imagenes"]
lista_etiquetas_prueba = datos_divididos["prueba"]["etiquetas"]

# Estructura de datos_divididos:
# {
#     "entrenamiento": {
#         "imagenes": ["ruta/a/imagen1.jpg", "ruta/a/imagen2.png", ...],
#         "etiquetas": ["persona1", "persona1", ...]
#     },
#     "prueba": {
#         "imagenes": ["ruta/a/imagen5.jpg", "ruta/a/imagen6.png", ...],
#         "etiquetas": ["persona2", "persona2", ...]
#     }
# }

# ISOMAP

In [17]:
def precalcular_isomap(datos_entrenamiento, n_componentes_isomap=10, n_vecinos_isomap=5):
    """
    Precalcula las ubicaciones de Isomap para las imágenes de entrenamiento.

    Args:
        datos_entrenamiento (dict): Diccionario con rutas de imágenes y etiquetas de entrenamiento.
        n_componentes_isomap (int): Número de componentes para Isomap.
        n_vecinos_isomap (int): Número de vecinos para Isomap.

    Returns:
        tuple: (ubicaciones_isomap, etiquetas_entrenamiento)
    """

    # Cargar imágenes de entrenamiento
    imagenes_entrenamiento = []
    for ruta in datos_entrenamiento["imagenes"]:
        img = Image.open(ruta).convert('L')
        imagenes_entrenamiento.append(np.array(img).flatten())
    imagenes_entrenamiento = np.array(imagenes_entrenamiento)

    # Isomap
    isomap = Isomap(n_components=n_componentes_isomap, n_neighbors=n_vecinos_isomap)
    ubicaciones_isomap = isomap.fit_transform(imagenes_entrenamiento)

    return ubicaciones_isomap, datos_entrenamiento["etiquetas"], isomap


In [18]:
# Seteamos variable para manejo mas simple, incluye etiquetas
datos_entrenamiento = datos_divididos["entrenamiento"]

In [19]:
# Precalculamos datos
ubicaciones_isomap, etiquetas_entrenamiento, isomap_model = precalcular_isomap(datos_entrenamiento)

In [12]:
# Guardar datos precalculados
datos_guardar = {
    "ubicaciones_isomap": ubicaciones_isomap,
    "etiquetas_entrenamiento": etiquetas_entrenamiento,
    "n_componentes_isomap": 10,  # Guardar hiperparámetros
    "n_vecinos_isomap": 5,
    "isomap_model": isomap_model  # Store the fitted Isomap model
}

In [20]:
# Archivo parametros modelo

archivo_params = "/content/drive/MyDrive/datos_isomap.pkl"

with open(archivo_params, "wb") as f:
    pickle.dump(datos_guardar, f)

In [ ]:
# Version nueva
def reconocer_cara_produccion(imagen_prueba, ruta_datos_precalculados=archivo_params):
    """
    Reconoce una cara usando datos precalculados de Isomap.

    Args:
        imagen_prueba (str): Ruta a la imagen de prueba.
        ruta_datos_precalculados (str): Ruta al archivo con los datos precalculados.

    Returns:
        list: Lista de las N etiquetas más cercanas y sus distancias.
    """

    # Cargar datos precalculados
    with open(ruta_datos_precalculados, "rb") as f:
        datos_cargados = pickle.load(f)

    ubicaciones_isomap = datos_cargados["ubicaciones_isomap"]
    etiquetas_entrenamiento = datos_cargados["etiquetas_entrenamiento"]
    # Get the pre-trained Isomap model
    isomap = datos_cargados["isomap_model"]

    # Cargar y mapear la imagen de prueba
    imagen_prueba_cargada = np.array(Image.open(imagen_prueba).convert('L')).flatten()

    # Transform the test image using the pre-trained model
    imagen_prueba_isomap = isomap.transform([imagen_prueba_cargada])

    # Calcular distancias y ranking
    distancias = euclidean_distances(imagen_prueba_isomap, ubicaciones_isomap)[0]
    indices_ordenados = np.argsort(distancias)[:3]
    ranking = [(etiquetas_entrenamiento[i], distancias[i]) for i in indices_ordenados]

    return ranking

In [ ]:
# Ejemplo de uso en producción
imagen_prueba = datos_divididos["prueba"]["imagenes"][50]
print(datos_divididos["prueba"]["etiquetas"][50])
ranking = reconocer_cara_produccion(imagen_prueba)
print(ranking)

Mauricio
[('Mauricio', np.float64(871.2282202464642)), ('Mauricio', np.float64(2400.977975380197)), ('Mauricio', np.float64(2862.2127745397192)), ('Mauricio', np.float64(2862.212774539722)), ('Mauricio', np.float64(4130.367088787949)), ('Mauricio', np.float64(4668.2972827933045)), ('Mauricio', np.float64(6177.613299640747)), ('José', np.float64(6434.254399807999)), ('Mauricio', np.float64(6573.955028137777)), ('José', np.float64(6690.849825896741)), ('José', np.float64(7847.270632749893)), ('José', np.float64(7995.576404384289)), ('Guadalupe', np.float64(8180.197631725395))]


In [ ]:
def reconocer_cara_produccion2(imagen_prueba, ruta_datos_precalculados=archivo_params):
    """
    Reconoce una cara usando datos precalculados de Isomap.

    Args:
        imagen_prueba (str): Ruta a la imagen de prueba.
        ruta_datos_precalculados (str): Ruta al archivo con los datos precalculados.

    Returns:
        list: Lista de las 3 etiquetas más cercanas y distintas, con sus distancias.
    """

    # Cargar datos precalculados (same as before)
    with open(ruta_datos_precalculados, "rb") as f:
        datos_cargados = pickle.load(f)

    ubicaciones_isomap = datos_cargados["ubicaciones_isomap"]
    etiquetas_entrenamiento = datos_cargados["etiquetas_entrenamiento"]
    isomap = datos_cargados["isomap_model"]

    # Cargar y mapear la imagen de prueba (same as before)
    imagen_prueba_cargada = np.array(Image.open(imagen_prueba).convert('L')).flatten()
    imagen_prueba_isomap = isomap.transform([imagen_prueba_cargada])

    # Calcular distancias (same as before)
    distancias = euclidean_distances(imagen_prueba_isomap, ubicaciones_isomap)[0]
    indices_ordenados = np.argsort(distancias)

    # Obtener las 3 etiquetas distintas más cercanas
    ranking_01 = []
    etiquetas_encontradas = set()  # Use a set to track distinct labels
    for i in indices_ordenados:
        etiqueta = etiquetas_entrenamiento[i]
        if etiqueta not in etiquetas_encontradas:
            ranking_01.append((etiqueta, distancias[i]))
            etiquetas_encontradas.add(etiqueta)
            if len(ranking_01) == 3:
                break

    return ranking

In [ ]:
print(ranking[0][0])

Noelia M


In [ ]:
# Ejemplo de uso en producción
imagen_prueba = datos_divididos["prueba"]["imagenes"][67]
print(f'Etiqueta: {datos_divididos["prueba"]["etiquetas"][67]}')
ranking = reconocer_cara_produccion2(imagen_prueba)
for i in range(len(ranking)):
  print(f"Candidato {i+1}: {ranking[i][0]} - Indice: {ranking[i][1]} ")

Etiqueta: Cristian
Candidato 1: Cristian - Indice: 1715.9601663120566 
Candidato 2: Alejandro - Indice: 3359.0499217868232 
Candidato 3: Daniel - Indice: 4354.524365185596 


In [ ]:
softmax

In [ ]:
procesar_archivos_en_carpetas(ruta_entrada, ruta_salida, img_size)

para cada archivo (imagen)
  para hp1 de 1 a 100
    para hp2 de 1 a 15
      isomap(imagen)
      softmax(imagen)
      error(imagen)
      registrar_HPS_error_minimo()